#### Se utilizó el cuaderno Representaciones_distribuidas

In [8]:
import sys
sys.path.append("../src")  

In [29]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from preprocesamiento import tokenizar, construir_vocabulario, generar_pares_skipgram
from skipgram import SkipGram

In [20]:
# Leemos el corpus
with open("../data/corpus.txt", "r", encoding="utf-8") as f:
    texto = f.read()

In [21]:
# Realizamos la tokenizacion
tokens = tokenizar(texto)
print(f"Corpus tokenizado: {len(tokens)} tokens")
print("Tokens:", tokens[:10])

Corpus tokenizado: 379598 tokens
Tokens: ['capítulo', 'primero', 'que', 'trata', 'de', 'la', 'condición', 'y', 'ejercicio', 'del']


In [22]:
# Construimos el vocabulario
vocab, word_to_ix, ix_to_word = construir_vocabulario(tokens)
V = len(vocab)
print(f"Tamaño del vocabulario: {V}")

Tamaño del vocabulario: 22904


In [24]:
window_size = 2
pares = generar_pares_skipgram(tokens, window_size)
print(f"Pares generados: {len(pares)}")
print("Ejemplo de 10 pares:", pares[:10])

Pares generados: 1518386
Ejemplo de 10 pares: [('capítulo', 'primero'), ('capítulo', 'que'), ('primero', 'capítulo'), ('primero', 'que'), ('primero', 'trata'), ('que', 'capítulo'), ('que', 'primero'), ('que', 'trata'), ('que', 'de'), ('trata', 'primero')]


In [28]:
#Convertir pares a indices
pares_idx = [(word_to_ix[t], word_to_ix[c]) for t, c in pares]
print("Ejemplo de 10 pares:", pares_idx[:10])

Ejemplo de 10 pares: [(3860, 17146), (3860, 17612), (17146, 3860), (17146, 17612), (17146, 21372), (17612, 3860), (17612, 17146), (17612, 21372), (17612, 6137), (21372, 17146)]


In [ ]:
embedding_dim = 100
model = SkipGram(V, embedding_dim)
optimizer = optim.SGD(model.parameters(), lr=0.025)
loss_fn = nn.BCEWithLogitsLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
epochs = 3
batch_size = 512
k = 5  # número de negativos por positivo

for epoch in range(epochs):
    total_loss = 0
    for u_batch, v_batch, labels in generar_batch(pares_idx, vocab_size, k, batch_size):
        u_batch = u_batch.to(device)
        v_batch = v_batch.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        scores = model(u_batch, v_batch)
        loss = loss_fn(scores, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")
